In [41]:
from scipy.optimize import minimize
import math
import numpy as np


In [60]:
def info_entropy(p, num_list):
    S = sum(num_list)
    N = len(num_list)
    Entropy = 0
    for n in num_list:
        n_pos = n*(1-p)
        n_neg = (S-n)*p/(N-1)
        P_pos = n_pos / (n_pos + n_neg)
        P_neg = n_neg / (n_pos + n_neg)
        Ei = - P_pos * math.log(P_pos, 2) - P_neg * math.log(P_neg, 2)
        # print(Ri)
        Pi = n / S # proba
        # print(Pi)
        Entropy = Entropy + Pi * Ei
        
    return -Entropy

In [76]:
p0 = [0.5]
num_list = [10, 20, 30, 40, 90, 2]
cons = ({'type': 'ineq',
         'fun': lambda x: np.array([x[0]-0.01]),
         'jac': lambda x: np.array([1.0])},
        {'type': 'ineq',
         'fun': lambda x: np.array([0.99-x[0]]),
         'jac': lambda x: np.array([-1.0])})
p = minimize(info_entropy, p0, args=(num_list), constraints=cons, method='SLSQP')

In [81]:
import json 

attr_dict_file = 'data/equal_processed_data/attr_to_attrvals.json'
vocab_dict_file = 'dataset/vocab/vocab_dict.json'
with open(attr_dict_file, 'r') as f:
    attr_dict = json.load(f)
with open(vocab_dict_file, 'r') as f:
    vocab_dict = json.load(f)
def get_negative_dict(attr_dict, vocab_dict):
    proba_negative_dict = {}
    for query, attr_list in attr_dict.items():
        proba_negative_dict[query] = {}
        proba_negative_dict[query]['attr_list'] = attr_list
        proba_list = []
        for attr in attr_list:
            proba_list.append(vocab_dict[attr])
            proba_negative_dict[query]['attr_freq'] = proba_list
            
    return proba_negative_dict

proba_negative_dict = get_negative_dict(attr_dict, vocab_dict)

In [86]:
import copy 
from scipy.optimize import minimize
import math
import numpy as np

dict_copy = copy.deepcopy(proba_negative_dict)
for query, attr_dict in dict_copy.items():
     num_list = attr_dict['attr_freq']
     p0 = [0.5] # 注意函数p是负例生成概率
     cons = ({'type': 'ineq',
          'fun': lambda x: np.array([x[0]-0.01]),
          'jac': lambda x: np.array([1.0])},
          {'type': 'ineq',
          'fun': lambda x: np.array([0.99-x[0]]),
          'jac': lambda x: np.array([-1.0])})
     p = minimize(info_entropy, p0, args=(num_list), constraints=cons, method='SLSQP')
     proba_negative_dict[query]['entropy_proba'] = p.x.item()
     

In [90]:
attr_save_file = 'data/equal_processed_data/proba_negative_dict.json'
with open(attr_save_file, 'w') as f:
    json.dump(proba_negative_dict, f, ensure_ascii=False, indent=4)